# Warping tutorial
## e_warping_and_filtering_and_phase_comp

##### May 2020
###### Eva Chamorro - Daniel Zitterbart - Julien Bonnel

## 1. Import packages

In [1]:
import os
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
%matplotlib widget
from matplotlib import interactive
from matplotlib.path import Path
from scipy.signal import hilbert
from scipy import interpolate
from scipy.fftpack import fft, ifft
from warping_functions import *
from time_frequency_analysis_functions import *
from bbox_select import *
import warnings
warnings.filterwarnings('ignore')

## 2. Load simulated signal

In [2]:
data = sio.loadmat(os.getcwd()+ '/sig_pek_for_warp.mat')


'''
    s_t: propagated modes in a Pekeris waveguide with parameters
    c1, c2, rho1, rho2: sound speed / density
        D: depth
        r: range
        zs, zr: source/receiver depth
    s_t_dec: same than s_t, except that time origin has been set for warping
    fs: sampling frequency
    
   
     NB: one can run optional_create_simulated_signal.m to generate another
     simulated signal
'''
# Select variables
s_t=data['s_t']
fs=data['fs']
s_t_dec=data['s_t_dec']
r=data['r']
c1=data['c1']

## 3. Process signal 

In [3]:
# The first sample of s_t_dec corresponds to time r/c1
# Make the signal shorter, no need to keep samples with zeros
N_ok=250
ir_ok=s_t_dec[:,0:N_ok]
print('Continue')

Continue


## 4. Source signal creation

In [4]:
X,IFLAW_source=fmpar(200,np.array([1,0.5]),np.array([100,0.15]),np.array([200,0.01]))


# source signal is a parabolic modulated FM signal
source=X*(IFLAW_source+1)
N_source=len(source)

print('Continue')

Continue


## 5. Propagated signal

In [5]:
# Propagated signal = time-convolution between source and impulse response = multiplication in the frequency domain
source_f=fft(source, N_ok)
ir_f=fft(ir_ok,N_ok)
s_f=np.transpose(source_f*ir_f)
s_ok=ifft(s_f,N_ok, axis=0) #propagated signal

# STFT computation
NFFT=1024
N_window=31 # you need a short window to see the modes
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
d=np.hamming(N_window)
d=d[:,np.newaxis]
tfr=tfrstft(s_ok,b,NFFT,d)
source_1=source[:,np.newaxis]
tfr_source=tfrstft(source_1,b,NFFT,d)

spectro=abs(tfr)**2
spectro_source=abs(tfr_source)**2


# Time and frequency axis of the original signal
time=np.arange(0,N_ok)/fs
freq=np.arange(0,NFFT)*fs/NFFT

print('The source is a non-linear (parabolic) FM signal')
print('The received signal is highly influenced by the source')


# Figure

plt.figure(figsize=(10,7))
plt.subplot(121)
plt.imshow(spectro_source, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Source signal')


plt.subplot(122)
plt.imshow(spectro,extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Received signal')
plt.show()



print('Looking at the received signal, you may think that the source is a linear FM downsweep.')
print('Although wrong, this is a good enough approximation for warping')
print('Let us illustrate this, continue')
print('')



The source is a non-linear (parabolic) FM signal
The received signal is highly influenced by the source


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Looking at the received signal, you may think that the source is a linear FM downsweep.
Although wrong, this is a good enough approximation for warping
Let us illustrate this, continue



## 6. Phase compensation

In [6]:
### The guess source will be a linear sweep with following parameters
L=130   ### length of the sweep in samples
f1=0.5  ### start frequency of the sweep (reduced frequency, i.e. f1=0.5*fs Hz)
f2=0.01 ### end frequency of the sweep (reduced frequency, i.e. f2=0.01*fs Hz)

source_est, IFLAW_est=fmlin(L,f1,f2) ### estimated source signal in the time domain
source_est_f=fft(source_est,N_ok,axis=0) ### estimated source signal in the frequency domain
phi=np.angle(source_est_f)                 ### phase of the estimated source signal in the frequency domain


### let's plot it on top of the received signal
print('As an example, the black line may be our best guess of the source signal')
print('Now, let us do phase compensation to transform our received signal')
print('into something that looks like the impulse response of the waveguide')

# Figure
plt.figure(figsize=(7,5))
plt.subplot(121)
plt.imshow(spectro_source, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Source signal')



# Figure
plt.subplot(122)
plt.imshow(spectro, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Received signal')
x1=0
x2=(L-1)/fs
y1=0.5*fs
y2=0.01*fs
plt.plot([0, x2[0,0]], [y1[0,0],y2[0,0]], linewidth=3, color='black')
plt.show()


print('Continue')



As an example, the black line may be our best guess of the source signal
Now, let us do phase compensation to transform our received signal
into something that looks like the impulse response of the waveguide


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue


### 6.1 Phase correction

In [7]:
# Phase correction
sig_prop_f=fft(s_ok,axis=0)
i=complex(0,1)
sig_rec_f=sig_prop_f*np.exp(-i*phi) # note that only source phase is deconvoluted
sig_rec_t=ifft(sig_rec_f,axis=0)  #Signal in time domain after source deconvolution


# Figure
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
d=np.hamming(N_window)
d=d[:,np.newaxis]
tfr_sig_rec=tfrstft(sig_rec_t,b,NFFT,d)

print('The result is not too bad, it indeed looks like an impulse response with modes')
print('Now let us warp!')

plt.figure()
plt.imshow(abs(tfr_sig_rec)**2, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0,fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Signal after phase compensation')
plt.show()

print('Continue')

The result is not too bad, it indeed looks like an impulse response with modes
Now let us warp!


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue


## 7. Warping

In [8]:
# The warped signal will be s_w

s_w, fs_w=warp_temp_exa(np.transpose(sig_rec_t),fs,r,c1)
M=len(s_w)

# STFT computation
N_window_w=301 # You need a long window to see the warped modes
wind=np.hamming(N_window_w)/np.linalg.norm(np.hamming(N_window_w))
wind=wind[:,np.newaxis]
b=np.arange(1,M+1)
b=b[np.newaxis,:]
tfr_w=tfrstft(s_w,b,NFFT,wind)
spectro_w=abs(tfr_w)**2

# Time and frequency axis of the warped signal      
time_w=np.arange(0,M)/fs_w
freq_w=np.arange(0,NFFT)*fs_w/NFFT # this is actually not exact, but precise enough for a rough example
print('Continue filtering')

Continue filtering


## 8. Filtering

In [9]:
## selection the spectro_w part to mask 
spectro_w_1=spectro_w[0:200,0:648]

print('Not too bad huh?')
print('Now click on the spectrogram to create the mask to filter one mode')
print('To do so, click twice on the spectrogram to create a line and continue to define the region you want to filter ')
print('(see c_warping_and_filtering.m if necessary)')
print(' ')


# Figure

#Let use bbox_select

interactive(True)
section= bbox_select(spectro_w_1)
print('Click disconect mpl and continue to mask the section selected')

Not too bad huh?
Now click on the spectrogram to create the mask to filter one mode
To do so, click twice on the spectrogram to create a line and continue to define the region you want to filter 
(see c_warping_and_filtering.m if necessary)
 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

Click disconect mpl and continue to mask the section selected


In [11]:
# create the mask of section
pts=section.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))
masque_1=np.double(mask)

# add the part masked to the total sprectogram array
masque_2=np.zeros_like(spectro_w[200:,:])
masque_3=np.zeros_like(spectro_w[0:200,648:])
masque=np.concatenate((masque_1,masque_3),axis=1)
masque=np.concatenate((masque,masque_2),axis=0)

# Note that the mask is applied on the STFT (not on the spectrogram)
mode_rtf_warp=masque*tfr_w
norm=1/NFFT/np.max(wind)
mode_temp_warp=np.real(np.sum(mode_rtf_warp,axis=0))*norm
mode=iwarp_temp_exa(mode_temp_warp,fs_w,r,c1,fs,N_ok)

print('Continue to see the result')

Continue to see the result


## 9. Verification

In [12]:
# you can estimate the dispersion curve by computing the
# frequency moment of the filtered mode TFR
a=hilbert(mode)
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
h=np.hamming(N_window)
h=h[:,np.newaxis]
mode_stft=tfrstft(a,b,NFFT,h)
mode_spectro=np.abs(mode_stft)**2
tm,D2=momftfr(mode_spectro,0,N_ok,time)

print('The red line is the estimated dispersion curve.')
print('You have to restrict it to a frequency band where it is relevant')
print('If the result is not satisfying, you must create a new filtering mask')
print('Let us assume that the result is ok.')
print('Now we need to undo the phase compensation to look at the true filtered mode')

#Figure
plt.figure()
plt.imshow(abs(tfr_sig_rec)**2, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low') 
plt.ylim([0,fs/2])
plt.xlim([0,1])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Signal after phase compensation and estimated dispersion curve')

plt.plot(tm[:,0],freq[0,:], linewidth=2, color='r')
plt.show()


print('Continue')

The red line is the estimated dispersion curve.
You have to restrict it to a frequency band where it is relevant
If the result is not satisfying, you must create a new filtering mask
Let us assume that the result is ok.
Now we need to undo the phase compensation to look at the true filtered mode


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue


## 10. Let's undo phase compensation

In [13]:
mode_f=fft(mode,axis=0)
i=complex(0,1)
mode_rec_f=mode_f*np.exp(i*phi)
mode_rec_t=ifft(mode_rec_f,axis=0) ### Mode in time domain after source re-convolution
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
d=np.hamming(N_window)
d=d[:,np.newaxis]
mode_rec_sp=tfrstft(mode_rec_t,b,NFFT,d)
mode_rec_sp=abs(mode_rec_sp)


### We need to undo the source deconvolution for the dispersion curve too
####### first we define the time-frequency law of our estimated source
f_source_est=IFLAW_est*fs
t_source_est=np.arange(0,len(IFLAW_est))/fs


###### we need it on the same frequency axis than the dispersion curves
f = interpolate.interp1d(f_source_est[:,0], t_source_est[0,:], bounds_error=False, fill_value=np.nan )
t_source_est_ok = f(freq[0,:]) 
tm_est_with_source=tm[:,0]+t_source_est_ok

print('The left subplot is the spectrogram of the filtered mode')
print('The right subplot shows the received signal and the estimated dispersion curve')
print('Now, let us compare your result with the true modes')
print('')

plt.figure(figsize=(7,5))
plt.subplot(121)
plt.imshow(mode_rec_sp, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low') 
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Filtered mode')

plt.subplot(122)
#plt.pcolormesh(time[0,:], freq[0,:], spectro)
plt.imshow(spectro, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low') 
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Original signal and estimated dispersion curve')


plt.plot(tm_est_with_source,freq[0,:],linewidth=3, color='black')

print('Continue')

The left subplot is the spectrogram of the filtered mode
The right subplot shows the received signal and the estimated dispersion curve
Now, let us compare your result with the true modes



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue


## 11. Verification

In [14]:
data = sio.loadmat(os.getcwd()+ '/sig_pek_and_modes_for_warp.mat')
r=data['r']
vg=data['vg']
c1=data['c1']
f_vg=data['f_vg']



### we need a few tricks to create the theoretical dispersion curves
#### first the dispersion curves of the impulse response

tm_theo_ir=r/vg-r/c1 ### range over group_speed minus correction for time origin
##### now we need to include the source law ....
f_source=IFLAW_source*fs

t_source=np.arange(0,len(IFLAW_source))/fs


##### but we need it on the same frequency axis than tm_theo_ir
f = interpolate.interp1d(f_source[0,:], t_source[0,:], bounds_error=False, fill_value=np.nan )
t_source_ok = f(f_vg[0,:]) 
t_source_ok_1=np.tile(t_source_ok,(5,1))
tm_theo_with_source=tm_theo_ir+t_source_ok_1

print('This is the same figure than before, except that')
print('the true dispersion curves are now in black.')
print('Remember that the estimated dispersion curve is ')
print('relevant only in the frequency band where it matches')
print('the estimated spectrogram.')
print(' ')

plt.figure()
#plt.pcolormesh(time[0,:], freq[0,:], spectro)
plt.imshow(spectro, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low') 
plt.ylim([0,fs/2])
plt.xlim([0,1.2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Original signal and estimated dispersion curve')
plt.plot(tm_theo_with_source[0,:], f_vg[0,:], 'black')
plt.plot(tm_theo_with_source[1,:], f_vg[0,:], 'black')
plt.plot(tm_theo_with_source[2,:], f_vg[0,:], 'black')
plt.plot(tm_theo_with_source[3,:], f_vg[0,:], 'black')
plt.plot(tm_est_with_source,freq[0,:], linewidth=3, color='r')
plt.show()


print('END')

This is the same figure than before, except that
the true dispersion curves are now in black.
Remember that the estimated dispersion curve is 
relevant only in the frequency band where it matches
the estimated spectrogram.
 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

END
